## <center>**Requirements Multi-class Classification**</center>

### **1. Construct the Task Prompt**

In [193]:
from utils.fileUtils import *

In [194]:
lines = readExcelToList('./promise_nfr.xlsx')

In [195]:
len(lines)

626

In [196]:
data_lines = []
for line in lines[1:]:
    label = line[3].strip()
    if label in ['PE','SE','US','O']:
        data_lines.append(line)

In [197]:
# column 2 for Req text
# column 3 for NFR sub class
len(data_lines)

249

In [198]:
import openai

In [199]:
openai.api_key = 'YOUR_OWN_API_KEY'

In [200]:
test_req = data_lines[89]
print(test_req[2],test_req[3])

Only adjusters can request recycled parts audit reports. No users without an adjuster role shall request recycled parts audits. SE


In [201]:
# Tag one quality label from (Usability, Security, Operational, Performance) for the following non-functional requirement statement.
prompt = '''Tag one quality label from (Usability, Security, Operational, Performance) for the following non-functional requirement statement.:\n\n\n'''

In [202]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt + test_req[2]}],
        temperature = 0,
        top_p = 1
    )

In [203]:
content = response['choices'][0]['message']['content']

In [204]:
content.strip()

'Security'

In [205]:
print(prompt + test_req[2])

Tag one quality label from (Usability, Security, Operational, Performance) for the following non-functional requirement statement.:


Only adjusters can request recycled parts audit reports. No users without an adjuster role shall request recycled parts audits.


### **2. Multi-class Classification on Requirements with ChatGPT**

In [206]:
def query_ChatGPT(req_str):
    # Tag one quality label from (Usability, Security, Operational, Performance) for the following non-functional requirement statement.:\n\n\n
    prompt = 'Tag one quality label from (Usability, Security, Operational, Performance) for the following non-functional requirement statement.:\n\n\n'
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [{"role": "user", "content": prompt + req_str}],
        temperature = 0,
        top_p = 1
    )
    content = response['choices'][0]['message']['content']
    label = content.strip()
    return label

In [207]:
import time
import logging
from datetime import datetime

# current time
now = datetime.now()
# current time string
time_str = now.strftime("%Y-%m-%d %H:%M:%S")

logging.basicConfig(level=logging.INFO,
                    filename='./{}.log'.format(time_str),
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [208]:
lines = readExcelToList('./promise_nfr.xlsx')

In [209]:
assert len(lines) == (625+1)

In [17]:
data_lines = []
for line in lines[1:]:
    label = line[3].strip()
    if label in ['PE','SE','US','O']:
        data_lines.append(line)

In [210]:
assert len(data_lines) == 249

In [ ]:
ans_list = []
pre_list = []
for line in data_lines:
    req_id = line[0].strip()
    logging.info(req_id)
    ans_label = line[3].strip()
    ans_list.append(ans_label)
    req_text = line[2].strip()
    while True:
        try:
            ans_content = query_ChatGPT(req_text)
            pre_list.append(ans_content)
            break
        except:
            time.sleep(3)

In [22]:
assert len(pre_list) == 249

249

In [23]:
len(set(pre_list))

8

In [24]:
set(pre_list)

{'Operational',
 'Operational.',
 'Performance',
 'Performance.',
 'Security',
 'Security.',
 'Usability',
 'Usability.'}

In [25]:
pre_list_l = []
for item in pre_list:
    if item.startswith('O'):
        pre_list_l.append('O')
    else:
        pre_list_l.append(item[:2].upper())

In [216]:
# dumpJson('./pre_list_raw.json',pre_list)

In [214]:
# dumpJson('./pre_list_clean.json',pre_list_l)

In [ ]:
# dumpJson('./ans_list.json',ans_list)

### **3. Leave-one-project-out cross-validation**

In [ ]:
# pre_list_l = loadJson('./pre_list_clean.json')
# ans_list = loadJson('./ans_list.json')

In [221]:
# if you have cleaned the ChatGPT responses with reference to the answer set, 
# set the postprocessed to True and continue to run the following evaluation codes.
postprocessed = False

In [222]:
assert postprocessed == True, "See the following note!"

AssertionError: See the following note!

**Note**: Before running the following evaluation codes, you should firstly clean the ChatGPT responses.

In [173]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

In [174]:
nfr_labels = ['PE','SE','US','O']

In [175]:
project_dict = {}
for idx,line in enumerate(data_lines):
    req_id = idx
    proj_id = int(line[1].strip())
    project_dict.setdefault(proj_id,[])
    project_dict[proj_id].append(idx)

In [176]:
project_num = len(project_dict.keys())
print('Project Num:', project_num)

Project Num: 15


In [177]:
# The shape of project_results is (# of projects, # of NFR subclass, three measures<P,R,F1>)
project_results = np.zeros((project_num,len(nfr_labels),3))

In [178]:
for proj_id in project_dict.keys():
    proj_ans_list = []
    proj_pre_list = []
    for idx in project_dict[proj_id]:
        ans = ans_list[idx]
        pre = pre_list_l[idx]
        proj_ans_list.append(ans)
        proj_pre_list.append(pre)
    report = precision_recall_fscore_support(proj_ans_list, proj_pre_list,labels=nfr_labels)
    # In the transposed matrix, each line corresponds a NFR subclass.
    # Columns correspond seperately to precision, recall, and F1 scores.
    report_matrix = np.transpose(np.asarray(report))[:,:-1]
    project_results[proj_id-1] = report_matrix

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.9/site-packages/skle

In [179]:
# project_results

In [180]:
# sum the measures of each NFR sub-class over 15 projects
np.sum(project_results,axis=0)

array([[11.77948718, 12.78039216, 11.87893218],
       [13.5       , 13.3       , 13.26797386],
       [10.75728291, 11.31282051, 10.84047619],
       [ 8.53418803,  8.29206349,  8.13492063]])

In [181]:
# Find the all zeros rows. That means some NFR subclass does not appear in a project
all_zeros = np.all(project_results == 0, axis=2)
# e.g. the element in (1,3) is True. That means the Operational(O) NFR requirements do not appear in Project 2.
print(all_zeros)

[[False False False  True]
 [False False False  True]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False  True False]
 [False False False False]
 [False  True  True False]
 [False False False  True]
 [False False False  True]
 [False False False False]
 [ True False False False]
 [False False False False]
 [False False False False]]


In [182]:
# The first arr includes project IDs (starts from 0). 
# The second arr includes the index of NFR sub-classes, nfr_labels = ['PE','SE','US','O'] 
np.where(all_zeros==True)

(array([ 0,  1,  6,  8,  8,  9, 10, 12]), array([3, 3, 2, 1, 2, 3, 3, 0]))

#### **3.1 Average performance of PE**

In [183]:
# compute the number of projects which do not include the PE requirements
label_idx = nfr_labels.index('PE')
PE_abs_num = np.count_nonzero(np.where(all_zeros==True)[1]==label_idx)
print(PE_abs_num)

1


In [184]:
# compute the average performance measures of PE over projects which include that type of NFR.
np.around(
    np.sum(project_results,axis=0)[label_idx]/(project_num-PE_abs_num),
    2
)

array([0.84, 0.91, 0.85])

#### **3.2 Average performance of SE**

In [185]:
# compute the number of projects which do not include the SE requirements
label_idx = nfr_labels.index('SE')
SE_abs_num = np.count_nonzero(np.where(all_zeros==True)[1]==label_idx)
print(SE_abs_num)

1


In [186]:
# compute the average performance measures of SE over projects which include that type of NFR.
np.around(
    np.sum(project_results,axis=0)[label_idx]/(project_num-SE_abs_num),
    2
)

array([0.96, 0.95, 0.95])

#### **3.3 Average performance of US**

In [187]:
# compute the number of projects which do not include the US requirements
label_idx = nfr_labels.index('US')
US_abs_num = np.count_nonzero(np.where(all_zeros==True)[1]==label_idx)
print(US_abs_num)

2


In [188]:
# compute the average performance measures of US over projects which include that type of NFR.
np.around(
    np.sum(project_results,axis=0)[label_idx]/(project_num-US_abs_num),
    2
)

array([0.83, 0.87, 0.83])

#### **3.4 Average performance of O**

In [189]:
# compute the number of projects which do not include the O requirements
label_idx = nfr_labels.index('O')
O_abs_num = np.count_nonzero(np.where(all_zeros==True)[1]==label_idx)
print(O_abs_num)

4


In [190]:
# compute the average performance measures of US over projects which include that type of NFR.
np.around(
    np.sum(project_results,axis=0)[label_idx]/(project_num-O_abs_num),
    2
)

array([0.78, 0.75, 0.74])